In [56]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from pathlib import Path

In [57]:
def load_json(dir):
    try:
        with open(dir) as f:
            data = json.load(f)
            
        return data
    
    except:
        return None

In [58]:
RESULT_DIR = "..\\result_summary"

In [59]:
def parse_file_nm(file_nm):
    mcts_info = None
    
    components = file_nm.split("\\")
    
    if "debug" in components:
        components.remove("debug")
    
    method = components[2]
    
    diff_cut = 1
    
    if 'am' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, _ = components
        
    elif 'mcts' in method:
        _, _, method, diff_cut, _, prob_type, num_probs, model_info, baseline_info, mcts_info, _ = components
        diff_cut = float(diff_cut.split("-")[-1])

    num_probs = num_probs.split("-")[0]
    num_probs = int(num_probs.split("_")[1])
    baseline_info = baseline_info.split("-")[-1]
    
    return method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut

In [60]:
for file in glob.glob(f"{RESULT_DIR}/**/*.json", recursive=True):
    result = load_json(file)

    avg_result = False
    for k, v in result.items():
        for k2, v2 in v.items():
            if 'result' in k2:
                avg_result = True
                break
    
    if not avg_result and len(result) != 102:
        print(len(result))
        print(file)
        
# need to do diff-0.75 and CVRP with N=50 for 4 and 6

In [80]:
# parse all json file named with "all_result_avg.json" from the RESULT_DIR

all_result = {}

for file in glob.glob(f"{RESULT_DIR}/**/all_result_avg.json", recursive=True):
    data = load_json(file)
    if "cross_test_result" in file or 'diff' in file:
        continue
    # print(file)
    method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut = parse_file_nm(file)
    
    if data is not None:
        
        if method not in all_result:
            all_result[method] = {}
        
        if prob_type not in all_result[method]:
            all_result[method][prob_type] = {}
            
        if num_probs not in all_result[method][prob_type]:
            all_result[method][prob_type][num_probs] = {}
        
        if mcts_info is None:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
            
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = data
                
        else:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
                
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = {}
                
            if mcts_info not in all_result[method][prob_type][num_probs][model_info][baseline_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info] = {}
                
            if diff_cut not in all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info][diff_cut] = data
                
    # break

In [81]:
def get_parital_result(method, prob_type, num_prob):       
    partial_result = {'model_info': [], 'baseline': [], 'mcts_info': [], 'epoch': [], 'score': [], 'score_std': [], 'runtime': [],  'runtime_std': [], 'diff_cut': []}
    
    def _append_epoch_result(model_info, baseline_info, epoch, mcts, epoch_score, epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut):
        partial_result['model_info'].append(model_info)
        partial_result['baseline'].append(baseline_info)
        partial_result['mcts_info'].append(mcts)
        partial_result['epoch'].append(epoch)                
        partial_result['score'].append(epoch_score)
        partial_result['score_std'].append(epoch_score_std)
        partial_result['runtime'].append(epoch_runtime)
        partial_result['runtime_std'].append(epoch_runtime_std)
        partial_result['diff_cut'].append(diff_cut)

    target_result = all_result[method][prob_type][num_prob]
    
    if "am" in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_epoch_results = target_result[model_info][baseline_info]
                
                for epoch in all_epoch_results:
                    current_epoch_result = all_epoch_results[epoch]['result_avg']
                    epoch_score = current_epoch_result['score']
                    epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                    epoch_runtime = current_epoch_result['runtime']
                    epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                    
                    _append_epoch_result(model_info, baseline_info, epoch, "am", epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, 1)
                    
    elif 'mcts' in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_mcts_results = target_result[model_info][baseline_info]
                
                for mcts_info in all_mcts_results:
                    all_diff_results = all_mcts_results[mcts_info]
                    
                    for diff_cut in all_diff_results:
                        all_epoch_results = all_diff_results[diff_cut]
                    
                        for epoch in all_epoch_results:
                            current_epoch_result = all_epoch_results[epoch]['result_avg']
                            epoch_score = current_epoch_result['score']
                            epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                            epoch_runtime = current_epoch_result['runtime']
                            epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                            
                            _append_epoch_result(model_info, baseline_info, epoch, mcts_info, epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut)                                         
            
    df = pd.DataFrame.from_dict(partial_result)
    df['activation'] = df['model_info'].apply(lambda x: x.split("-")[5])
    df['enc_layers'] = df['model_info'].apply(lambda x: x.split('-')[2])
    df = df.drop(columns=['model_info'])
    
    df['train_score'] = df.apply(lambda x: x['epoch'].split("-")[1].split("=")[1], axis=1).astype(float)
    df['epoch'] =  df.apply(lambda x: x['epoch'].split("-")[0].split("=")[1], axis=1).astype(int)
    
    if 'mcts' in method:
        df['cpuct'] = df['mcts_info'].apply(lambda x: x.split("-")[2].split('_')[1])
        df['ns'] = df['mcts_info'].apply(lambda x: x.split("-")[0].split('_')[1])
        df['cpuct'] = df['cpuct'].astype(float)
        df['ns'] = df['ns'].astype(int)
    
    elif 'am' in method:
        df['cpuct'] = 0
        df['ns'] = 0
        
    df = df.drop(columns=['mcts_info'])
    

        
    return df

In [82]:
def plot_bar_result(base_df, baseline, activation, prob_type, num_prob, plot_dev=False, hue='cpuct'):
    if prob_type == 'tsp':
        y_ranges = {20: (3.75, 3.95), 50: (5.7, 5.875), 100: (7.95, 8.25)}
        
    elif prob_type == 'cvrp':
        y_ranges = {20: (6.1, 7.5), 50: (9, 12), 100: (7.95, 8.25)}
        
    _df = base_df[(base_df['baseline'] == baseline) & (base_df['activation'] == activation)]
    
    # drop rows where the mcts_info is 0.8
    _df = _df[_df['cpuct'] != '0.8']
    
    # sort by the train score
    _df = _df.sort_values(by=['epoch', 'cpuct', 'ns']).reset_index(drop=True)
    _df['score_std'] = _df['score_std'].astype(float)
    
    _df['cpuct'] = _df['cpuct'].astype(object)
    _df['cpuct'] = _df['cpuct'].fillna('am')
    
    _df['ns'] = _df['ns'].astype(object)
    _df['ns'] = _df['ns'].fillna('am')
    
    plt.figure(figsize=(12, 7))
    
    ax = sns.barplot(data=_df, x=_df['epoch'], y='score', hue=hue)
    
    if plot_dev:
        bar_coords = [(rect.get_x(), rect.get_y(), rect.get_width(), rect.get_height()) for rect in ax.patches]
                
        for i, (x, y, w, h) in enumerate(bar_coords):
            x_pos = x + w / 2
            y_top = y + h + 0.01
            
            plt.errorbar(x=x_pos, y=y_top, yerr=_df['score_std'][i], fmt='none', color='black', capsize=4)
    
    plt.title(f"{prob_type}_{num_prob}_{baseline}_{activation}")
    plt.ylim(*y_ranges[num_prob])
    plt.legend()
    
    path = Path(f"../result_image/bars")
    
    if not path.exists():
        path.mkdir(parents=True)
        
    
    plt.savefig(f"{path}/{prob_type}_{num_prob}_{baseline}_{activation}.png")
    plt.show()

In [83]:
# tsp_20 = pd.concat([get_parital_result('am', 'tsp', 20), get_parital_result('mcts_v2', 'tsp', 20)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_20, _baseline, _activation, 'tsp', 20, hue='cpuct')

In [84]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100)])
# tsp_100.to_csv("../result_csv/tsp_100.csv")
tsp_100.sort_values(by=['score']).head(15)

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
28,val,299,7.948579,0.072763,1.272957,1.469262,0.75,swiglu,6,7.86777,1.1,500
23,val,299,7.950269,0.071354,2.555101,7.452994,0.75,swiglu,6,7.86777,1.1,1000
15,val,299,7.951494,0.074433,1.785637,0.808511,0.10,swiglu,6,7.86777,1.1,100
20,val,299,7.952069,0.071879,20.467145,147.939535,0.10,swiglu,6,7.86777,1.1,1000
21,val,299,7.952187,0.072126,29.216151,86.161708,0.25,swiglu,6,7.86777,1.1,1000
2,mean,299,7.952278,0.073115,0.684153,0.066761,0.50,swiglu,6,7.90623,1.1,100
1,mean,299,7.952360,0.073393,2.045356,0.412602,0.25,swiglu,6,7.90623,1.1,100
0,mean,299,7.952381,0.073451,1.561934,0.549741,0.10,swiglu,6,7.90623,1.1,100
24,val,299,7.952483,0.072638,0.866759,1.669436,1.00,swiglu,6,7.86777,1.1,1000
29,val,299,7.952672,0.072723,0.576197,0.406174,1.00,swiglu,6,7.86777,1.1,500


In [85]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        df = pd.concat([get_parital_result('am', env_type, N), get_parital_result('mcts', env_type, N)])
        df.to_csv(f"../result_csv/{env_type}_{N}.csv", index=False)

In [86]:
# load tsp and N=20

def get_pivot_table(env_type, N, epoch=299, enc_layers=6, diff_cut=0.75, index=['epoch', 'activation', 'baseline', 'ns'], refine_to_str=True):
    df = pd.read_csv(f"../result_csv/{env_type}_{N}.csv")
    cond1 = df['epoch'] == epoch
    cond2 = df['enc_layers'] == enc_layers
    cond3 = df['diff_cut'] == diff_cut 
    cond4 = df['diff_cut'] == 1
    df = df[cond1 & cond2 & (cond3 | cond4)]
    pivot_table = df.pivot_table(index=index, values=['score', 'score_std', 'runtime', 'runtime_std'])
    # # reindex the column order as to score mean, score std, runtime mean, runtime std
    pivot_table = pivot_table.reindex(columns=['score', 'score_std', 'runtime', 'runtime_std'])
    if refine_to_str:
        pivot_table['score'] = pivot_table.apply(lambda x: f"{round(x['score'], 5):.4f} ± {round(x['score_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['score_std'], axis=1)
    
        pivot_table['runtime'] = pivot_table.apply(lambda x: f"{round(x['runtime'], 4):.3f} ± {round(x['runtime_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['runtime_std'], axis=1)
        
    return pivot_table

In [87]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=True, diff_cut=0.1) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100                
                                        score         runtime
epoch activation baseline ns                                 
299   relu       mean     0     7.9852 ± 0.08    0.121 ± 0.00
                 val      0     8.0481 ± 0.08    0.120 ± 0.00
      swiglu     mean     0     7.9562 ± 0.07    0.123 ± 0.00
                          100   7.9532 ± 0.07    0.938 ± 0.28
                          500   7.9552 ± 0.07    4.337 ± 9.55
                          1000  7.9547 ± 0.07   8.887 ± 45.01
                 val      0     7.9551 ± 0.07    0.124 ± 0.00
                          100   7.9531 ± 0.07    1.051 ± 0.41
                          500   7.9546 ± 0.07   5.215 ± 16.91
                          1000  7.9523 ± 0.07  10.667 ± 74.80

In [96]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=True, diff_cut=0.25) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df

100                
                                        score         runtime
epoch activation baseline ns                                 
299   relu       mean     0     7.9852 ± 0.08    0.121 ± 0.00
                 val      0     8.0481 ± 0.08    0.120 ± 0.00
      swiglu     mean     0     7.9562 ± 0.07    0.123 ± 0.00
                          100   7.9532 ± 0.07    1.180 ± 0.21
                          500   7.9544 ± 0.07    5.739 ± 7.98
                          1000  7.9543 ± 0.07  12.532 ± 41.43
                 val      0     7.9551 ± 0.07    0.124 ± 0.00
                          100   7.9537 ± 0.07    1.361 ± 0.25
                          500   7.9578 ± 0.07    6.963 ± 8.41
                          1000  7.9523 ± 0.07  15.041 ± 43.92

In [89]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=True, diff_cut=0.5) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df

100               
                                        score        runtime
epoch activation baseline ns                                
299   relu       mean     0     7.9852 ± 0.08   0.121 ± 0.00
                          100   7.9854 ± 0.08   0.774 ± 0.06
                          500   7.9817 ± 0.08   3.406 ± 2.17
                          1000  7.9819 ± 0.08  7.197 ± 10.29
                 val      0     8.0481 ± 0.08   0.120 ± 0.00
                          100   8.0465 ± 0.08   0.775 ± 0.06
                          500   8.0654 ± 0.09   3.650 ± 2.09
                          1000  8.0566 ± 0.08   7.621 ± 9.40
      swiglu     mean     0     7.9562 ± 0.07   0.123 ± 0.00
                          100   7.9532 ± 0.07   0.499 ± 0.04
                          500   7.9542 ± 0.07   1.681 ± 1.16
                          1000  7.9543 ± 0.07   3.405 ± 5.95
                 val      0     7.9551 ± 0.07   0.124 ± 0.00
                          100   7.9541 ± 0.07   0.556 ± 0.04
                          500   7.9584 ± 0.07   2.081 ± 1.45
                          1000  7.9529 ± 0.07   4.243 ± 6.65

In [90]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=True, diff_cut=0.75) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df

100              
                                        score       runtime
epoch activation baseline ns                               
299   relu       mean     0     7.9852 ± 0.08  0.121 ± 0.00
                 val      0     8.0481 ± 0.08  0.120 ± 0.00
      swiglu     mean     0     7.9562 ± 0.07  0.123 ± 0.00
                          100   7.9541 ± 0.07  0.358 ± 0.02
                          500   7.9553 ± 0.07  0.728 ± 0.67
                          1000  7.9543 ± 0.07  1.277 ± 3.42
                 val      0     7.9551 ± 0.07  0.124 ± 0.00
                          100   7.9554 ± 0.07  0.394 ± 0.03
                          500   7.9506 ± 0.07  0.925 ± 0.94
                          1000  7.9514 ± 0.07  1.711 ± 4.56

In [91]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=True, diff_cut=1) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df

100              
                                        score       runtime
epoch activation baseline ns                               
299   relu       mean     0     7.9852 ± 0.08  0.121 ± 0.00
                 val      0     8.0481 ± 0.08  0.120 ± 0.00
      swiglu     mean     0     7.9562 ± 0.07  0.123 ± 0.00
                          100   7.9541 ± 0.07  0.314 ± 0.01
                          500   7.9554 ± 0.07  0.496 ± 0.25
                          1000  7.9547 ± 0.07  0.729 ± 1.12
                 val      0     7.9551 ± 0.07  0.124 ± 0.00
                          100   7.9547 ± 0.07  0.317 ± 0.01
                          500   7.9527 ± 0.07  0.576 ± 0.41
                          1000  7.9525 ± 0.07  0.867 ± 1.67

In [92]:
tsp_results = [get_pivot_table('cvrp', N, refine_to_str=True, diff_cut=0.1) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100              
                                         score       runtime
epoch activation baseline ns                                
299   relu       mean     0     16.4418 ± 2.99  0.140 ± 0.00
                 val      0     16.4463 ± 3.00  0.139 ± 0.00
      swiglu     mean     0     16.4043 ± 3.09  0.142 ± 0.00
                          100   16.4143 ± 3.08  0.218 ± 0.01
                          500   16.4130 ± 3.11  0.547 ± 0.22
                          1000  16.4088 ± 3.12  0.974 ± 0.93
                 val      0     16.3575 ± 3.04  0.142 ± 0.00
                          100   16.3666 ± 3.03  0.232 ± 0.01
                          500   16.3583 ± 3.05  0.647 ± 0.29
                          1000  16.3572 ± 3.04  1.168 ± 1.18

In [93]:
tsp_results = [get_pivot_table('cvrp', N, refine_to_str=True, diff_cut=0.25) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100                
                                         score         runtime
epoch activation baseline ns                                  
299   relu       mean     0     16.4418 ± 2.99    0.140 ± 0.00
                 val      0     16.4463 ± 3.00    0.139 ± 0.00
      swiglu     mean     0     16.4043 ± 3.09    0.142 ± 0.00
                          100   16.4242 ± 3.09    2.854 ± 0.63
                          500   16.4106 ± 3.11  16.915 ± 20.97
                          1000  16.4100 ± 3.11  34.895 ± 87.57
                 val      0     16.3575 ± 3.04    0.142 ± 0.00
                          100   16.3839 ± 3.05    2.730 ± 0.61
                          500   16.3561 ± 3.06  15.386 ± 15.60
                          1000  16.3558 ± 3.04  31.816 ± 66.79

In [94]:
tsp_results = [get_pivot_table('cvrp', N, refine_to_str=True, diff_cut=0.5) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100               
                                         score        runtime
epoch activation baseline ns                                 
299   relu       mean     0     16.4418 ± 2.99   0.140 ± 0.00
                 val      0     16.4463 ± 3.00   0.139 ± 0.00
      swiglu     mean     0     16.4043 ± 3.09   0.142 ± 0.00
                          100   16.4164 ± 3.09   0.910 ± 0.10
                          500   16.4118 ± 3.10   4.370 ± 2.99
                          1000  16.4100 ± 3.11  8.889 ± 13.80
                 val      0     16.3575 ± 3.04   0.142 ± 0.00
                          100   16.3802 ± 3.02   0.899 ± 0.11
                          500   16.3530 ± 3.05   4.338 ± 2.80
                          1000  16.3531 ± 3.03  8.700 ± 11.07

In [95]:
tsp_results = [get_pivot_table('cvrp', N, refine_to_str=True, diff_cut=0.75) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100               
                                         score        runtime
epoch activation baseline ns                                 
299   relu       mean     0     16.4418 ± 2.99   0.140 ± 0.00
                 val      0     16.4463 ± 3.00   0.139 ± 0.00
      swiglu     mean     0     16.4043 ± 3.09   0.142 ± 0.00
                          100   16.4145 ± 3.07   0.441 ± 0.08
                          500   16.4146 ± 3.11   1.578 ± 2.71
                          1000  16.4120 ± 3.12  3.125 ± 11.77
                 val      0     16.3575 ± 3.04   0.142 ± 0.00
                          100   16.3680 ± 3.02   0.459 ± 0.08
                          500   16.3575 ± 3.06   1.658 ± 1.89
                          1000  16.3573 ± 3.04   3.256 ± 8.14

In [97]:
tsp_results = [get_pivot_table('cvrp', N, refine_to_str=True, diff_cut=1) for N in [100]]
all_results_df = pd.concat(tsp_results, keys=[100],  axis=1)
all_results_df


100              
                                         score       runtime
epoch activation baseline ns                                
299   relu       mean     0     16.4418 ± 2.99  0.140 ± 0.00
                 val      0     16.4463 ± 3.00  0.139 ± 0.00
      swiglu     mean     0     16.4043 ± 3.09  0.142 ± 0.00
                          100   16.4143 ± 3.08  0.218 ± 0.01
                          500   16.4130 ± 3.11  0.547 ± 0.22
                          1000  16.4088 ± 3.12  0.974 ± 0.93
                 val      0     16.3575 ± 3.04  0.142 ± 0.00
                          100   16.3666 ± 3.03  0.232 ± 0.01
                          500   16.3583 ± 3.05  0.647 ± 0.29
                          1000  16.3572 ± 3.04  1.168 ± 1.18

In [78]:
tsp_results = [get_pivot_table('tsp', N, diff_cut=0.75) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                           50   \
                                        score       runtime          score   
epoch activation baseline ns                                                 
299   relu       mean     0     3.8492 ± 0.09  0.026 ± 0.00  5.7361 ± 0.06   
                          100   3.8486 ± 0.09  0.043 ± 0.00  5.7375 ± 0.06   
                          500   3.8491 ± 0.09  0.059 ± 0.00  5.7345 ± 0.06   
                          1000  3.8491 ± 0.09  0.077 ± 0.01  5.7339 ± 0.06   
                 val      0     3.8489 ± 0.08  0.027 ± 0.00  5.7386 ± 0.06   
                          100   3.8483 ± 0.08  0.038 ± 0.00  5.7310 ± 0.07   
                          500   3.8483 ± 0.08  0.044 ± 0.00  5.7291 ± 0.07   
                          1000  3.8483 ± 0.08  0.052 ± 0.01  5.7301 ± 0.06   
      swiglu     mean     0     3.8464 ± 0.08  0.027 ± 0.00  5.7267 ± 0.07   
                          100   3.8462 ± 0.08  0.045 ± 0.00  5.7248 ± 0.07   
                          500   3.8461 ± 0.08  0.065 ± 0.01  5.7260 ± 0.07   
                          1000  3.8461 ± 0.08  0.089 ± 0.02  5.7257 ± 0.07   
                 val      0     3.8482 ± 0.09  0.027 ± 0.00  5.7405 ± 0.07   
                          100   3.8474 ± 0.09  0.047 ± 0.00  5.7377 ± 0.07   
                          500   3.8478 ± 0.09  0.069 ± 0.01  5.7377 ± 0.07   
                          1000  3.8478 ± 0.09  0.093 ± 0.02  5.7373 ± 0.07   

                                                        100                 
                                     runtime          score        runtime  
epoch activation baseline ns                                                
299   relu       mean     0     0.060 ± 0.00  7.9852 ± 0.08   0.121 ± 0.00  
                          100   0.205 ± 0.01  7.9854 ± 0.08   0.627 ± 0.06  
                          500   0.592 ± 0.23  7.9826 ± 0.08   2.588 ± 1.82  
                          1000  1.051 ± 0.85  7.9807 ± 0.08   5.509 ± 9.14  
                 val      0     0.060 ± 0.00  8.0481 ± 0.08   0.120 ± 0.00  
                          100   0.194 ± 0.01  8.0465 ± 0.08   0.648 ± 0.08  
                          500   0.533 ± 0.21  8.0687 ± 0.09   2.897 ± 2.03  
                          1000  0.959 ± 0.88  8.0564 ± 0.08   5.915 ± 9.45  
      swiglu     mean     0     0.061 ± 0.00  7.9562 ± 0.07   0.123 ± 0.00  
                          100   0.173 ± 0.01  7.9523 ± 0.07   0.621 ± 0.13  
                          500   0.458 ± 0.17  7.9530 ± 0.07   2.586 ± 3.31  
                          1000  0.799 ± 0.62  7.9539 ± 0.07  5.553 ± 18.05  
                 val      0     0.060 ± 0.00  7.9551 ± 0.07   0.124 ± 0.00  
                          100   0.192 ± 0.01  7.9536 ± 0.07   0.679 ± 0.12  
                          500   0.550 ± 0.19  7.9630 ± 0.07   3.114 ± 3.52  
                          1000  0.961 ± 0.70  7.9522 ± 0.07  6.532 ± 17.42

In [79]:
cvrp_results = [get_pivot_table('cvrp', N, diff_cut=0.75) for N in [20, 50, 100]]
all_results_df = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                            50   \
                                        score       runtime           score   
epoch activation baseline ns                                                  
299   relu       mean     0     6.4097 ± 0.75  0.032 ± 0.00  10.8050 ± 1.65   
                          100   6.4010 ± 0.76  0.151 ± 0.01  10.8225 ± 1.59   
                          500   6.4065 ± 0.75  0.398 ± 0.13  10.8192 ± 1.60   
                          1000  6.4073 ± 0.76  0.685 ± 0.45  10.8087 ± 1.63   
                 val      0     6.4553 ± 0.80  0.032 ± 0.00  10.8634 ± 1.65   
                          100   6.4452 ± 0.83  0.176 ± 0.01  10.8718 ± 1.61   
                          500   6.4449 ± 0.84  0.489 ± 0.15  10.8651 ± 1.65   
                          1000  6.4479 ± 0.80  0.831 ± 0.47  10.8774 ± 1.67   
      swiglu     mean     0     6.4231 ± 0.73  0.032 ± 0.00  10.7940 ± 1.70   
                          100   6.4228 ± 0.75  0.141 ± 0.01  10.7747 ± 1.71   
                          500   6.4230 ± 0.73  0.346 ± 0.08  10.7878 ± 1.72   
                          1000  6.4292 ± 0.74  0.562 ± 0.23  10.7847 ± 1.67   
                 val      0     6.4201 ± 0.72  0.032 ± 0.00  10.8402 ± 1.48   
                          100   6.4020 ± 0.74  0.154 ± 0.01  10.8420 ± 1.53   
                          500   6.4055 ± 0.73  0.422 ± 0.10  10.8336 ± 1.49   
                          1000  6.4101 ± 0.72  0.686 ± 0.28  10.8456 ± 1.53   

                                                         100                  
                                     runtime           score         runtime  
epoch activation baseline ns                                                  
299   relu       mean     0     0.073 ± 0.00  16.4418 ± 2.99    0.140 ± 0.00  
                          100   0.473 ± 0.03  16.4627 ± 3.00    1.326 ± 0.12  
                          500   1.737 ± 0.81  16.4596 ± 2.99    6.652 ± 3.91  
                          1000  3.258 ± 2.95  16.4387 ± 2.96  13.633 ± 17.61  
                 val      0     0.073 ± 0.00  16.4463 ± 3.00    0.139 ± 0.00  
                          100   0.466 ± 0.03  16.4750 ± 3.08    1.409 ± 0.13  
                          500   1.773 ± 0.78  16.4494 ± 3.02    7.079 ± 3.34  
                          1000  3.475 ± 3.65  16.4468 ± 2.98  14.504 ± 14.40  
      swiglu     mean     0     0.075 ± 0.00  16.4043 ± 3.09    0.142 ± 0.00  
                          100   0.464 ± 0.05  16.4190 ± 3.10    1.253 ± 0.15  
                          500   1.742 ± 1.02  16.4169 ± 3.10    6.293 ± 5.36  
                          1000  3.260 ± 3.71  16.4117 ± 3.11  12.700 ± 23.47  
                 val      0     0.075 ± 0.00  16.3575 ± 3.04    0.142 ± 0.00  
                          100   0.517 ± 0.04  16.3950 ± 2.97    1.201 ± 0.12  
                          500   1.959 ± 1.02  16.3475 ± 3.05    5.947 ± 4.08  
                          1000  3.683 ± 3.64  16.3492 ± 3.02  12.072 ± 16.35

In [89]:
tsp_results = [get_pivot_table('tsp', N, index=['activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                 \
                         score       runtime           score       runtime   
     activation                                                              
tsp  relu        3.8487 ± 0.09  0.124 ± 0.02   5.7340 ± 0.06  0.907 ± 0.83   
     swiglu      3.8470 ± 0.08  0.152 ± 0.05   5.7318 ± 0.07  0.818 ± 0.66   
cvrp relu        6.4255 ± 0.79  0.828 ± 0.68  10.8442 ± 1.63  2.672 ± 3.46   
     swiglu      6.4149 ± 0.74  0.721 ± 0.42  10.8147 ± 1.61  2.765 ± 4.12   

                            100                  
                          score         runtime  
     activation                                  
tsp  relu         8.0190 ± 0.08    4.225 ± 6.96  
     swiglu       7.9551 ± 0.07    3.881 ± 6.80  
cvrp relu        16.4556 ± 3.02  11.354 ± 16.35  
     swiglu      16.3895 ± 3.06   9.527 ± 16.50

In [ ]:
tsp_results = [get_pivot_table('tsp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

In [232]:
tsp_results = [get_pivot_table('tsp', N, index=['baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50             \
                  score score_std   runtime runtime_std      score score_std   
     baseline                                                                  
tsp  mean      3.847532  0.085914  0.143579    0.034008   5.730290  0.064384   
     val       3.848140  0.084904  0.132662    0.030068   5.735478  0.065808   
cvrp mean      6.414754  0.746359  0.704385    0.498716  10.801845  1.663723   
     val       6.425673  0.775440  0.843840    0.600933  10.857036  1.570981   

                                           100                       \
                runtime runtime_std      score score_std    runtime   
     baseline                                                         
tsp  mean      0.849227    0.719525   7.968395  0.076000   3.837999   
     val       0.876634    0.774389   8.005644  0.077479   4.267926   
cvrp mean      2.603224    3.584254  16.435670  3.053555  10.466180   
     val       2.833637    3.987187  16.409435  3.034862  10.415026   

                           
              runtime_std  
     baseline              
tsp  mean        7.168259  
     val         6.591142  
cvrp mean       18.769996  
     val        14.081803

In [98]:
tsp_results = [get_pivot_table('tsp', N, index=['baseline', 'ns']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['baseline', 'ns']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                 \
                            score       runtime           score       runtime   
     baseline ns                                                                
tsp  mean     0     3.8478 ± 0.09  0.026 ± 0.00   5.7314 ± 0.06  0.060 ± 0.00   
              100   3.8464 ± 0.08  0.031 ± 0.00   5.7267 ± 0.07  0.149 ± 0.00   
              500   3.8464 ± 0.08  0.033 ± 0.00   5.7256 ± 0.07  0.228 ± 0.12   
              1000  3.8464 ± 0.08  0.034 ± 0.00   5.7256 ± 0.07  0.311 ± 0.54   
     val      0     3.8486 ± 0.08  0.027 ± 0.00   5.7395 ± 0.07  0.060 ± 0.00   
              100   3.8482 ± 0.09  0.031 ± 0.00   5.7405 ± 0.07  0.159 ± 0.01   
              500   3.8481 ± 0.09  0.034 ± 0.00   5.7405 ± 0.07  0.246 ± 0.14   
              1000  3.8481 ± 0.09  0.037 ± 0.00   5.7405 ± 0.07  0.350 ± 0.48   
cvrp mean     0     6.4164 ± 0.74  0.032 ± 0.00  10.7995 ± 1.67  0.074 ± 0.00   
              100   6.4260 ± 0.74  0.133 ± 0.01  10.7858 ± 1.68  0.312 ± 0.04   
              500   6.4276 ± 0.74  0.294 ± 0.15  10.7918 ± 1.67  0.847 ± 0.85   
              1000  6.4329 ± 0.74  0.472 ± 0.48  10.7915 ± 1.68  1.480 ± 3.20   
     val      0     6.4377 ± 0.76  0.032 ± 0.00  10.8518 ± 1.57  0.074 ± 0.00   
              100   6.4240 ± 0.76  0.168 ± 0.02  10.8304 ± 1.48  0.382 ± 0.05   
              500   6.4193 ± 0.72  0.406 ± 0.31  10.8376 ± 1.47  1.202 ± 1.31   
              1000  6.4185 ± 0.73  0.661 ± 0.91  10.8367 ± 1.48  2.166 ± 5.03   

                               100                 
                             score        runtime  
     baseline ns                                   
tsp  mean     0      7.9707 ± 0.08   0.122 ± 0.00  
              100    7.9541 ± 0.07   0.358 ± 0.02  
              500    7.9553 ± 0.07   0.728 ± 0.67  
              1000   7.9543 ± 0.07   1.277 ± 3.42  
     val      0      8.0016 ± 0.08   0.122 ± 0.00  
              100    7.9554 ± 0.07   0.394 ± 0.03  
              500    7.9506 ± 0.07   0.925 ± 0.94  
              1000   7.9514 ± 0.07   1.711 ± 4.56  
cvrp mean     0     16.4231 ± 3.04   0.141 ± 0.00  
              100   16.4145 ± 3.07   0.441 ± 0.08  
              500   16.4146 ± 3.11   1.578 ± 2.71  
              1000  16.4120 ± 3.12  3.125 ± 11.77  
     val      0     16.4019 ± 3.02   0.140 ± 0.00  
              100   16.3680 ± 3.02   0.459 ± 0.08  
              500   16.3575 ± 3.06   1.658 ± 1.89  
              1000  16.3573 ± 3.04   3.256 ± 8.14

In [233]:
tsp_results = [get_pivot_table('tsp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50   \
                             score score_std   runtime runtime_std      score   
     activation baseline                                                        
tsp  relu       mean      3.848909  0.087351  0.135863    0.024499   5.735429   
                val       3.848468  0.084752  0.111687    0.013510   5.732479   
     swiglu     mean      3.846156  0.084476  0.151295    0.043516   5.725151   
                val       3.847812  0.085056  0.153637    0.046626   5.738476   
cvrp relu       mean      6.406097  0.755201  0.753307    0.647176  10.815637   
                val       6.444959  0.815983  0.901911    0.708723  10.872692   
     swiglu     mean      6.423411  0.737517  0.655462    0.350257  10.788053   
                val       6.406387  0.734897  0.785769    0.493144  10.841380   

                                                                100            \
                         score_std   runtime runtime_std      score score_std   
     activation baseline                                                        
tsp  relu       mean      0.059666  0.954732    0.826665   7.983006  0.078750   
                val       0.065083  0.860328    0.836371   8.054898  0.083024   
     swiglu     mean      0.069102  0.743721    0.612386   7.953784  0.073250   
                val       0.066533  0.892941    0.712407   7.956390  0.071933   
cvrp relu       mean      1.617335  2.615675    3.151902  16.455791  3.008325   
                val       1.638581  2.728584    3.759984  16.455378  3.040988   
     swiglu     mean      1.710110  2.590773    4.016606  16.415550  3.098785   
                val       1.503381  2.938689    4.214389  16.363492  3.028735   

                                                 
                            runtime runtime_std  
     activation baseline                         
tsp  relu       mean       4.136831    7.754827  
                val        4.312788    6.167008  
     swiglu     mean       3.539167    6.581690  
                val        4.223063    7.015277  
cvrp relu       mean      11.087819   18.449787  
                val       11.620718   14.246272  
     swiglu     mean       9.844541   19.090205  
                val        9.209334   13.917334

In [38]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                           50   \
                             score       runtime          score   
     ns   activation                                              
tsp  0    relu        3.849 ± 0.09  0.109 ± 0.00   5.737 ± 0.06   
          swiglu      3.847 ± 0.08  0.111 ± 0.00   5.734 ± 0.07   
     100  relu        3.849 ± 0.09  0.094 ± 0.00   5.734 ± 0.06   
          swiglu      3.847 ± 0.08  0.106 ± 0.00   5.731 ± 0.07   
     250  relu        3.848 ± 0.09  0.130 ± 0.01   5.734 ± 0.06   
          swiglu      3.847 ± 0.08  0.149 ± 0.02   5.731 ± 0.07   
     500  relu        3.849 ± 0.09  0.127 ± 0.02   5.732 ± 0.06   
          swiglu      3.847 ± 0.08  0.165 ± 0.05   5.732 ± 0.07   
     1000 relu        3.849 ± 0.09  0.159 ± 0.06   5.732 ± 0.06   
          swiglu      3.847 ± 0.08  0.231 ± 0.16   5.731 ± 0.07   
cvrp 0    relu        6.433 ± 0.78  0.131 ± 0.00  10.834 ± 1.65   
          swiglu      6.422 ± 0.73  0.133 ± 0.00  10.817 ± 1.59   
     100  relu        6.423 ± 0.79  0.398 ± 0.08  10.847 ± 1.60   
          swiglu      6.412 ± 0.75  0.359 ± 0.05  10.808 ± 1.62   
     250  relu        6.419 ± 0.78  0.889 ± 0.44  10.854 ± 1.62   
          swiglu      6.407 ± 0.74  0.804 ± 0.27  10.822 ± 1.62   
     500  relu        6.426 ± 0.79  1.048 ± 0.74  10.842 ± 1.62   
          swiglu      6.414 ± 0.73  0.902 ± 0.47  10.811 ± 1.60   
     1000 relu        6.428 ± 0.78  1.672 ± 2.13  10.843 ± 1.65   
          swiglu      6.420 ± 0.73  1.405 ± 1.31  10.815 ± 1.60   

                                               100                  
                            runtime          score         runtime  
     ns   activation                                                
tsp  0    relu         0.230 ± 0.00   8.017 ± 0.08    0.450 ± 0.00  
          swiglu       0.236 ± 0.00   7.956 ± 0.07    0.455 ± 0.00  
     100  relu         0.368 ± 0.04   8.016 ± 0.08    1.248 ± 0.22  
          swiglu       0.341 ± 0.03   7.953 ± 0.07    1.177 ± 0.21  
     250  relu         0.956 ± 0.45   8.020 ± 0.08    3.997 ± 2.37  
          swiglu       0.853 ± 0.35   7.956 ± 0.07    3.670 ± 2.57  
     500  relu         1.119 ± 0.82   8.024 ± 0.08    5.240 ± 6.31  
          swiglu       1.001 ± 0.70   7.958 ± 0.07    4.782 ± 5.96  
     1000 relu         1.864 ± 2.85   8.018 ± 0.08  10.188 ± 25.91  
          swiglu       1.662 ± 2.23   7.953 ± 0.07   9.322 ± 25.24  
cvrp 0    relu         0.277 ± 0.00  16.444 ± 2.99    0.524 ± 0.00  
          swiglu       0.278 ± 0.00  16.381 ± 3.06    0.526 ± 0.00  
     100  relu         0.950 ± 0.15  16.469 ± 3.04    2.623 ± 0.45  
          swiglu       0.991 ± 0.19  16.407 ± 3.04    2.316 ± 0.50  
     250  relu         2.068 ± 0.99  16.468 ± 3.12  15.025 ± 13.29  
          swiglu       2.140 ± 1.23  16.397 ± 3.08  12.681 ± 10.18  
     500  relu         3.631 ± 3.45  16.454 ± 3.01  12.840 ± 12.76  
          swiglu       3.788 ± 4.34  16.382 ± 3.07  10.755 ± 13.84  
     1000 relu        6.434 ± 12.68  16.443 ± 2.97  25.759 ± 55.24  
          swiglu      6.627 ± 14.82  16.381 ± 3.06  21.355 ± 58.00

In [58]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
tsp_results = tsp_results.reset_index()
ns_col_name = tsp_results.columns[0]
ns_col_name

('ns', '')

In [84]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        corr_coef = get_pivot_table(env_type, N, index=['ns'], refine_to_str=False).reset_index().corr().loc['ns', 'score']
        print(f"{env_type}_{N}: {corr_coef}")

tsp_20: -0.11810629992614363
tsp_50: -0.7213754639076688
tsp_100: 0.14677544871966403
cvrp_20: 0.13159901236379598
cvrp_50: 0.02364275470398175
cvrp_100: -0.4732575291038821


In [70]:
def corr(ns, score):
    return np.corrcoef(ns, score)[0, 1]

for N in [20, 50, 100]:
    ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
    score = tsp_results.loc[:, (N, 'score')]
    print(f"N={N}: ", corr(ns, score))


N=20:  -0.02571148582075159
N=50:  -0.513745201132636
N=100:  0.010411342828905632


C:\Users\admin\AppData\Local\Temp\ipykernel_11920\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_11920\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_11920\1406123094.py:5: FutureWarning: The behavi

In [1]:
import os
from PIL import Image
from IPython.display import Image as Img
from IPython.display import display

def generate_gif(path, name):
    img_list = os.listdir(path)
    img_list = [path + '/' + x for x in img_list]
    images = [Image.open(x) for x in img_list]
    
    im = images[0]
    im.save(f'{name}.gif', save_all=True, append_images=images[1:], loop=0xff, duration=250)
    # loop 반복 횟수
    # duration 프레임 전환 속도 (500 = 0.5초)
    return Img(url=f'{name}.gif')

In [2]:
mcst_images_path = '../debug/plot/tsp/50/78/mcts'

generate_gif(mcst_images_path, 'tsp-50-75-mcts')

In [3]:
mcst_images_path = '../debug/plot/tsp/50/78/am'

generate_gif(mcst_images_path, 'tsp-50-75-am')

In [ ]:
def load_model_str_format(act, base, env_type, N):
    str_mcts = f"../result_summary/mcts/diff-0.75/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    str_am = f"../result_summary/am/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    return str_mcts, str_am

all_prob_result = {}

for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        indiv_result_dict = {'method': [], 'score': [], 'prob_num': []}
        for baseline in ['mean', 'val']:
            for act in ['relu', 'swiglu']:
                str_mcts, str_am = load_model_str_format(act, baseline, env_type, N)
                
                mcts_path = Path(str_mcts)
                
                for file in mcts_path.glob("**/*.json"):              
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        # find the number in file.name which comes after 'ns='
                        ns_num = int(str(file).split('\\')[9].split('-')[0].split('_')[1])
                        method = f"mcts-{ns_num}-{act}-{baseline}"    
                        
                        if ns_num == 250:
                            continue
                          
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])

                    else:
                        continue
                    
                for file in Path(str_am).glob("**/*.json"):
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        method = f"am-{act}-{baseline}"
                        
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])
                    else:
                        continue
                    
        all_prob_result[f"{env_type}_{N}"] = pd.DataFrame.from_dict(indiv_result_dict).pivot_table(index=['prob_num'], columns=['method'], values=['score'])

all_prob_result

In [ ]:
# define a paired t-test function for two pandas series
from scipy.stats import ttest_rel
def paired_ttest(s1, s2):
    return ttest_rel(s1, s2)


In [ ]:
from itertools import combinations

p_value_low_results = {}

for problem, df in all_prob_result.items():
    problem_p_value = {}
    
    for col1, col2 in combinations(df.columns, 2):
        split_col1 = col1[1].split("-")
        split_col2 = col2[1].split("-")
        
        # if split_col1[0] != split_col2[0] and split_col1[-1] == split_col2[-1] and split_col1[-2] == split_col2[-2]:
        #     problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
        
        if split_col1[0] != split_col2[0]:
            problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
    
    # sort the problem_p_value by the p-value
    problem_p_value = {k: v for k, v in sorted(problem_p_value.items(), key=lambda item: item[1])}    
    
    # leave only 1 lowest p-value records from problem_p_value
    first_record = list(problem_p_value.items())[0]
    problem_p_value = {first_record[0]: round(first_record[1], 4)}
    
    # problem_p_value = {k: v for k, v in problem_p_value.items() if v < 0.05}
    
    p_value_low_results[problem] = problem_p_value

p_value_low_results    
    

In [ ]:
def load_model_str_format(act, base, env_type, N):
    str_mcts = f"../result_summary/mcts/diff-0.75/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    str_am = f"../result_summary/am/pretrained_result/{env_type}/N_{N}-B_64/shared_mha-128-6-32-4-{act}-10-0.0001/1562-1-{base}"
    return str_mcts, str_am

all_prob_result = {}

for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        indiv_result_dict = {'method': [], 'score': [], 'prob_num': []}
        for baseline in ['mean', 'val']:
            for act in ['relu', 'swiglu']:
                str_mcts, str_am = load_model_str_format(act, baseline, env_type, N)
                
                mcts_path = Path(str_mcts)
                
                for file in mcts_path.glob("**/*.json"):              
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        # find the number in file.name which comes after 'ns='
                        ns_num = int(str(file).split('\\')[9].split('-')[0].split('_')[1])
                        method = f"mcts-{ns_num}-{act}-{baseline}"    
                        
                        if ns_num == 250:
                            continue
                          
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])

                    else:
                        continue
                    
                for file in Path(str_am).glob("**/*.json"):
                    if 'epoch=299' in file.name:
                        result_loaded = load_json(str(file))
                        method = f"am-{act}-{baseline}"
                        
                        for prob_num, val in result_loaded.items():
                            if prob_num == 'average' or prob_num == 'std':
                                continue
                            
                            indiv_result_dict['prob_num'].append(prob_num)
                            indiv_result_dict['method'].append(method)
                            indiv_result_dict['score'].append(val['score'])
                    else:
                        continue
                    
        all_prob_result[f"{env_type}_{N}"] = pd.DataFrame.from_dict(indiv_result_dict).pivot_table(index=['prob_num'], columns=['method'], values=['score'])

all_prob_result

In [ ]:
# define a paired t-test function for two pandas series
from scipy.stats import ttest_rel
def paired_ttest(s1, s2):
    return ttest_rel(s1, s2)


In [ ]:
from itertools import combinations

p_value_low_results = {}

for problem, df in all_prob_result.items():
    problem_p_value = {}
    
    for col1, col2 in combinations(df.columns, 2):
        split_col1 = col1[1].split("-")
        split_col2 = col2[1].split("-")
        
        # if split_col1[0] != split_col2[0] and split_col1[-1] == split_col2[-1] and split_col1[-2] == split_col2[-2]:
        #     problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
        
        if split_col1[0] != split_col2[0]:
            problem_p_value[col1[1], col2[1]] = paired_ttest(df[col1], df[col2]).pvalue
    
    # sort the problem_p_value by the p-value
    problem_p_value = {k: v for k, v in sorted(problem_p_value.items(), key=lambda item: item[1])}    
    
    # leave only 1 lowest p-value records from problem_p_value
    first_record = list(problem_p_value.items())[0]
    problem_p_value = {first_record[0]: round(first_record[1], 4)}
    
    # problem_p_value = {k: v for k, v in problem_p_value.items() if v < 0.05}
    
    p_value_low_results[problem] = problem_p_value

p_value_low_results    
    